# Year in Review &ndash; 2022

Interesting stats about ground stations on the [SatNOGS Network](https://network.satnogs.org).

&rarr; Skip past the setup to the [Statistics](#Statistics) section.

**Work in Progress**

# Ground Stations database setup

In [1]:
%load_ext sql

In [2]:
%sql sqlite:///../data/stations.db

'Connected: @../data/stations.db'

First thing is to see whuzzup with this (SQlite) database.

In [3]:
%%sql
CREATE TABLE IF NOT EXISTS stations (
    id PRIMARY KEY,
    altitude REAL,
    antenna TEXT CHECK(json_valid(antenna)),  -- array of objects
    client_version TEXT,
    created TEXT,       --  YYYY-mm-ddTHH:MMZ
    description TEXT,
    last_seen TEXT,     --  YYYY-mm-ddTHH:MMZ
    lat REAL,
    lng REAL,
    min_horizon REAL,
    name TEXT,
    observations INT,
    qthlocator TEXT,
    status TEXT,
    target_utilization INT,
    rotator INT         -- 0, 1, or 2 axis rotation
    );

-- Then, add rows from the stations.json.  This was done via CSV import, but it should
-- be configured to directly ingest from the JSON in order to keep the DB up to date.


 * sqlite:///../data/stations.db
Done.
Done.


[]

The main table `stations` is just a copy of the data returned by the SatNOGS Network API endpoint https://network.satnogs.org/api/stations


TODO: Automatically update the database from the API.

## Rotator information

The configuration for a ground station does not include machine-readable information about the antenna _mount_ -- how many axes of motion the antenna is moved about.
An omni-directional antenna affixed to a mast has 0 axes, while an azimuth + elevation rotator has 2 axes.
A single axis is also possible.

Notice from the schema for `stations` that there is one extra column, `rotator`, which will hold this information.
The problem is that we need to figure out the proper value to place in that field.

The `antenna` field holds a JSON array of objects holding information about the antennas attached to the particular station.

For example, the [main Valpo !WIREDlab station #834](https://network.satnogs.org/stations/834/) is configured with two antennas combined by a duplexer into a single RTL-SDR Blog v3 receiver.
The JSON array is then:

```json
"antenna": [
        {
            "frequency": 430000000,
            "frequency_max": 470000000,
            "band": "UHF",
            "antenna_type": "cross-yagi",
            "antenna_type_name": "Cross Yagi"
        },
        {
            "frequency": 135000000,
            "frequency_max": 150000000,
            "band": "VHF",
            "antenna_type": "cross-yagi",
            "antenna_type_name": "Cross Yagi"
        }
    ]
```

For our purposes here (inferring a correct `rotator` value), we focus on the object key `antenna_type`.

Extract the first item of the `antenna` array and summarize how many stations are using each type.

In [4]:
%%sql
-- Histogram of antenna types
SELECT
    json_extract(antenna, "$[0].antenna_type") as type,
    count(antenna) as num_type
FROM stations
GROUP BY type
ORDER BY num_type DESC;

 * sqlite:///../data/stations.db
Done.


type,num_type
turnstile,358
yagi,316
quadrafilar,290
vertical,249
cross-yagi,229
None,204
dipole,180
v-dipole,153
discone,140
eggbeater,139


Given these types of antennas, we can make a pretty good guess at which stations will have a rotator or not.
Omni-directional antenna types, e.g. turnstile and vertical, have no use for a rotator, while directional antennas such as yagi and helical likely would have a rotator.

Update the database to add these guesses.

In [5]:
guesses = (
    (0, "turnstile"),
    (2, "yagi"),
    (0, "quadrafilar"),
    (0, "vertical"),
    (2, "cross-yagi"),
    (0, "dipole"),
    (0, "v-dipole"),
    (0, "discone"),
    (0, "eggbeater"),
    (0, "other omni"),
    (2, "helical"),
    (0, "ground"),
    (0, "lindenblad"),
    (2, "other direct"),
    (2, "parabolic"),
    (0, "patch"),
    (0, "paralindy"),
)

for r, name in guesses:
    %sql UPDATE stations SET rotator = {r} \
         WHERE json_extract(antenna, "$[0].antenna_type") = "{name}";


 * sqlite:///../data/stations.db
358 rows affected.
 * sqlite:///../data/stations.db
316 rows affected.
 * sqlite:///../data/stations.db
290 rows affected.
 * sqlite:///../data/stations.db
249 rows affected.
 * sqlite:///../data/stations.db
229 rows affected.
 * sqlite:///../data/stations.db
180 rows affected.
 * sqlite:///../data/stations.db
153 rows affected.
 * sqlite:///../data/stations.db
140 rows affected.
 * sqlite:///../data/stations.db
139 rows affected.
 * sqlite:///../data/stations.db
91 rows affected.
 * sqlite:///../data/stations.db
80 rows affected.
 * sqlite:///../data/stations.db
69 rows affected.
 * sqlite:///../data/stations.db
38 rows affected.
 * sqlite:///../data/stations.db
35 rows affected.
 * sqlite:///../data/stations.db
18 rows affected.
 * sqlite:///../data/stations.db
10 rows affected.
 * sqlite:///../data/stations.db
5 rows affected.


Next up, recall that the station's name can contain useful information (FooName - fixed) along with the free-form Description text.

Set up a text search virtual table so we can look for text patterns:

In [6]:
%%sql
-- Setup for searching name, description fields for various text.
-- This is to find stations with different rotators than what may
-- be implied by their declared antenna_type(s)
CREATE VIRTUAL TABLE IF NOT EXISTS text 
USING FTS5(id, name, description);

INSERT OR REPLACE INTO text
SELECT id, name, description from stations;

 * sqlite:///../data/stations.db
Done.
2604 rows affected.


[]

Now let's look for stations where the **name** or **description** contain "fixed".

In [7]:
%%sql
-- Modify the rotator column to better reflect what the description claims
SELECT DISTINCT
    text.id,
    text.name,
    text.description,
    stations.antenna
FROM text, stations
WHERE text.id = stations.id
AND stations.rotator = 2
AND text MATCH 'fixed'
ORDER BY text.id;

 * sqlite:///../data/stations.db
Done.


id,name,description,antenna
66,KE8FZT - UHF,"Arrow UHF Corner Reflector, Fixed 20 Deg Elevation & Pointed 45 Deg NorthEast, AMSAT LNA,, MFJ Duplexer, Raspberry Pi3b","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":148000000},{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000}]"
212,KE8FZT - VHF,"Arrow VHF Corner Reflector, Fixed 20 Deg Elevation & Pointed 45 Deg NorthEast, LNA, Raspberry Pi3b","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":146000000}]"
1351,7el 70cm fixed Yagi,Restricted viewing angle is approximately 230 to 330 degrees.,"[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000}]"
1787,NCG,"Double bi-quad (13.46 dB) fixed position directional antenna looking e (W passes will not be seen well), altitude 48 deg, LNA (gain 20 dB noise 0.36dB), Lime, RPi 4","[{""antenna_type"":""other direct"",""antenna_type_name"":""Other Directional"",""band"":""VHF, UHF"",""frequency"":130000000,""frequency_max"":600000000}]"
1810,NCG2,"V-dipole fixed, looking 90 deg E, LNA (gain 20dB noise 0.8dB), Lime, RPi","[{""antenna_type"":""other direct"",""antenna_type_name"":""Other Directional"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":148000000}]"
1868,HB9FXX S-Band,My little s-band station: 4G feed on fixed dish pointing roughly at 300° (WNW) and 30-45° elevation-> VLNA13 -> Coax -> 1.5GHz LPF -> HackRF SDR -> RPi 4 8GB. The HackRF uses a Leo Bodnar GPSDO mini reference. On a first floor balcony with mostly west opening. Not a terrible location but the best I can do living in a flat.,"[{""antenna_type"":""parabolic"",""antenna_type_name"":""Parabolic"",""band"":""L, S"",""frequency"":1650000000,""frequency_max"":2400000000}]"
1950,KG7ZVV Satnogs,Fixed (no rotator) 10 element UHF Yagi. Visibility to only portion of sky towards Southwest. Apartments and building on other side.,"[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000}]"
2080,USU GAS Yagi Fixed SW,"Utah State University Get Away Special Team's second ground station. Wimo X-Quad (linear), No LNA currently. Fixed pointing SW at 45 deg elevation.","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":440000000}]"
2107,JH4XSY-pi,Pi4 + Airspy Mini + 3ele(145MHz)/6ele(435MHz) YAGI Fixed beam to the SouthWest.+ LNA,"[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":148000000},{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":420000000,""frequency_max"":450000000}]"
2246,HB9FXX S-Band 2,"Fixed dish at 300°, pointing like 30-40° up","[{""antenna_type"":""parabolic"",""antenna_type_name"":""Parabolic"",""band"":""L, S"",""frequency"":1650000000,""frequency_max"":2500000000}]"


It looks like all the stations matching the query have directional antennas but have them mounted in various fixed orientations.
Eyeballing the descriptions, the antennas are at 20 to 45 degree elevation angles.

Update the `stations` table with this new information.

In [8]:
%%sql
UPDATE stations
SET rotator = 0
WHERE id IN (66, 212, 1351, 1787, 1810, 1868, 1950, 2080, 2107, 2246, 2263,     
             2372, 2430, 2501, 2650);

 * sqlite:///../data/stations.db
15 rows affected.


[]

Now look for text that indicates a rotator model.
Popular units are the Yaesu G-5500 and the Alfa SPID.

In [9]:
%%sql
-- Find another pattern that implies a rotator
SELECT DISTINCT
    text.id,
    text.name,
    text.description,
    stations.antenna,
    stations.rotator
FROM text, stations
WHERE text.id = stations.id
AND text MATCH '5500 OR spid'
AND rotator <> 2
ORDER BY text.id;


 * sqlite:///../data/stations.db
Done.


id,name,description,antenna,rotator
888,CCERES,Directionnal WIMO VHF 10 elements and WIMO UHF 18 elements crossed yagi antennas + circular polarization coupler + UHF MVV70cm LNA + Diplexer 2m/70cm + SDR Airspy R2. Rotator : Yaesu G-5500 + GS-232B.Actually no have LNA for VHF antenna.,"[{""antenna_type"":""quadrafilar"",""antenna_type_name"":""Quadrafilar"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":440000000},{""antenna_type"":""cross-yagi"",""antenna_type_name"":""Cross Yagi"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":150000000}]",0
2500,Wolfgang HB9RYZ,Swiss Amateur Radio Station (JN47FE). Building up a brand new Ground Station in Q1/2022 with two Wimo X-Quad Antennas for 2m and 70cm including pre-amps and a SPID-RAS rotor.Temp.: running a Diamond vertical X-50NA (2m/70cm) on top of my roof.www.hb9ryz.ch,"[{""antenna_type"":""vertical"",""antenna_type_name"":""Vertical"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":148000000},{""antenna_type"":""vertical"",""antenna_type_name"":""Vertical"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000}]",0


And update the `rotator` field.
Note that this was tagged with `rotator = 0` since we only looked at the _first_ item in the `antenna` JSON array.

In [10]:
%%sql
UPDATE stations SET rotator = 2 WHERE id IN (888);

 * sqlite:///../data/stations.db
1 rows affected.


[]

Keep searching for keywords in the descriptions.

In [11]:
%%sql
SELECT DISTINCT
    text.id,
    text.name,
    text.description,
    stations.antenna,
    stations.rotator
FROM text, stations
WHERE text.id = stations.id
AND text MATCH 'azimuth'
ORDER BY text.id;


 * sqlite:///../data/stations.db
Done.


id,name,description,antenna,rotator
357,ce3vna,estacion completa de radioaficionados con rotor de azimuth y elevacion,"[{""antenna_type"":""ground"",""antenna_type_name"":""Ground Plane"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":148000000},{""antenna_type"":""ground"",""antenna_type_name"":""Ground Plane"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":440000000}]",0
1146,Ham Radio Ground Station,Ground station project specifically designed for communication with small satellites in Low Earth Orbit using VHF and UHF amateur radio frequencies and provide hands-on educational projects in the worldThe ground station can autonomously track satellites by using these equipments : VHF / UHF amateur radio transceiver (Amateur bands 144-16 MHZ and 430-440 MHZ)Rotator ControllerAzimuth and elevation rotators UHF and VHF Yagi antennas (Amateur bands 144-16 MHZ and 430-440 MHZ)Modems...,[],2
1775,Vi3w Roy4l,"Dual-Band Log Periodic Antenna or LPDA (log-periodic dipole array) VHF/UHF. 2 Meter gain is 6.8 dbd (8.9 dBi) and 440 Gain is 7 dBd (9 dBi). Set up on a 360° Rotator, Azimuth only with a set angle of 30°. The Azimuth changes from time to time when manually scheduling for Satellites on the Polar Plot. Rpi 3B+, RTL-SDR + RTL-LNA. Trying this all out and we'll see what happens, 1st Station on the Island.July: Added a Rotator Wifi controller on the LAN/WAN to control the Rotator on/off site.","[{""antenna_type"":""other direct"",""antenna_type_name"":""Other Directional"",""band"":""VHF"",""frequency"":136000000,""frequency_max"":148000000},{""antenna_type"":""other direct"",""antenna_type_name"":""Other Directional"",""band"":""UHF"",""frequency"":400000000,""frequency_max"":429000000},{""antenna_type"":""other direct"",""antenna_type_name"":""Other Directional"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000}]",2
2221,Francis Ground Station,Antenna array of 2 elements yagi for UHF Antenna dish and S-Band The Antenna array is attached to the edge of the dish of the S-Band Antenna The Antenna has a servo motor that controls azimuth and another for elevation,[],2


Two more stations with rotators.

AND a station with an azimuth-only axis!

In [12]:
%%sql
UPDATE stations SET rotator = 2 WHERE id IN (357, 1146, 2221);

-- And an azimuth-only station!
UPDATE stations SET rotator = 1 WHERE id IN (1775);

 * sqlite:///../data/stations.db
3 rows affected.
1 rows affected.


[]

Keep looking.  Searching for "azimuth" yielded nice results, so search for "elevation" next.

In [13]:
%%sql
SELECT DISTINCT
    text.id,
    text.name,
    text.description,
    stations.antenna,
    stations.rotator
FROM text, stations
WHERE text.id = stations.id
AND text MATCH 'elevation'
AND stations.rotator <> 0
ORDER BY text.id;


 * sqlite:///../data/stations.db
Done.


id,name,description,antenna,rotator
146,RIYADH2,31 Oct 2020 moved down street - 90 degree elevation crossed Yagi+LNA13 feb 2021 - wire antenna back - UHF LNA27 feb 2021 - short cross yagi looking up25 Sep 2022 - new location down the road - just monopole and no LNA - 3 stories22 Oct 2022 - UHF Yagi looking straight up with LNA,"[{""antenna_type"":""cross-yagi"",""antenna_type_name"":""Cross Yagi"",""band"":""UHF"",""frequency"":400000000,""frequency_max"":470000000}]",2
484,DParabole,Paris La Villette.Antenna is currently blocked at 90deg elevation,"[{""antenna_type"":""helical"",""antenna_type_name"":""Helical"",""band"":""L"",""frequency"":1500000000,""frequency_max"":1750000000}]",2
1146,Ham Radio Ground Station,Ground station project specifically designed for communication with small satellites in Low Earth Orbit using VHF and UHF amateur radio frequencies and provide hands-on educational projects in the worldThe ground station can autonomously track satellites by using these equipments : VHF / UHF amateur radio transceiver (Amateur bands 144-16 MHZ and 430-440 MHZ)Rotator ControllerAzimuth and elevation rotators UHF and VHF Yagi antennas (Amateur bands 144-16 MHZ and 430-440 MHZ)Modems...,[],2
1312,ESOC UHF,"437MHz Wimo cross yagi, phasing cable combiner, NooElec NESDR SMArt SMA 100700 RTL-SDR. V3.01 Satnogs Rotator, 3D printed elevation shafts, Raspberry Pi 3b. Radio Club + Cybernetics Club at ESA Space Operation Centre, Darmstadt.","[{""antenna_type"":""cross-yagi"",""antenna_type_name"":""Cross Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000}]",2
1423,OLIVIER F6OBT,"Rpi 4 + 1 Yagis antenna VHF 3 elements direction N-E, 1 Yagis antenna VHF 3 elements direction N-W and 1 Yagis antenna UHF 5 elements direction N-W, 1 Yagis antenna UHF 5 elements direction N-E. All antennas have an elevation angle of 40°.","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":152000000},{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":400000000,""frequency_max"":500000000}]",2
1466,PE0SAT-12,"HP NC6400 Laptop, Intel T7200 | Debian Bullseye (amd64) | SDR RX: AirSPY R2 | Antenna: DK7ZB 9 Element Yagi Pointing bearing 315 / elevation 75.","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":400000000,""frequency_max"":470000000}]",2
1640,VA7EEX - UHF,"Log periodic antenna with a 60 degree elevation, pointed southwest.","[{""antenna_type"":""other direct"",""antenna_type_name"":""Other Directional"",""band"":""UHF"",""frequency"":400000000,""frequency_max"":512000000}]",2
1710,EU1AEM,"In maintenance now!3-element yagi, vertical polarization, directed to east (100°), elevation 25° -> Uputronix 433-438 MHz filtred preamp -> RTL-SDR v3 -> Raspberry Pi 4.Please, do not use this station for west passes. Antenna is directed to east.Use station #2029 for west passes.","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":433000000,""frequency_max"":439000000}]",2
1789,jchem,"RPi4 + preamp 20dB + sdrrtl-v3, with VHF-4el.-horizontal-yagi pointing NE at 35° elevation.(if UHF: )two UHF-Yagi_6-el, 180d apart at 40d elevation, di. NE-SW.","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":148000000}]",2
2029,EU1AEM,"In maintenance now!3-element yagi, vertical polarization, directed to west (270°), elevation 30° -> Uputronix 433-438 MHz filtred preamp -> RTL-SDR v3 -> Raspberry Pi 4.Please, do not use this station for east passes. Antenna is directed to west.Use station #1710 for east passes.","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":433000000,""frequency_max"":439000000}]",2


... and found some more stations with directional antennas but with fixed pointing.

In [14]:
%%sql
-- More no-rotator stations
UPDATE stations SET rotator = 0 WHERE id IN (146, 484, 1423, 1466, 1640, 1710, 1789, 2029)

 * sqlite:///../data/stations.db
8 rows affected.


[]

It is interesting to note the stations with capabilites that are different than presumed.
Such a diversity of setups!

How about looking for a word meaning an exception such as "only"?

In [15]:
%%sql
SELECT DISTINCT
    text.id,
    text.name,
    text.description,
    stations.antenna,
    stations.rotator
FROM text, stations
WHERE text.id = stations.id
AND text MATCH 'only'
AND stations.rotator <> 0
ORDER BY text.id;


 * sqlite:///../data/stations.db
Done.


id,name,description,antenna,rotator
36,oe8rke,"2022-04-03: station close being reopend after rotor replacement2019-12-19: Still experiance issues with rotor - PLEASE ONLY EMERGENCY RECEPTIONS UNTIL ROTOR IS BACK, TNX!2019-12-10: Shutdown to service rotor again (stall issue), offsite location in the alps with low noise, operating with pstrotator hamlib interface","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000},{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""VHF"",""frequency"":144000000,""frequency_max"":146000000},{""antenna_type"":""helical"",""antenna_type_name"":""Helical"",""band"":""S"",""frequency"":2300000000,""frequency_max"":2450000000},{""antenna_type"":""helical"",""antenna_type_name"":""Helical"",""band"":""L"",""frequency"":1230000000,""frequency_max"":1290000000}]",2
87,M0IEB/M,"15.04.2018, Testing software with temporary home rig location, 23.04.2018, Home rig set, added LNA and filter for 145M band. Enjoy! 26.04.2018, Rig won't get 137M, 07.05.2018, Updated to 0.6.1, In case it was not clear, this station will not receive NOAA. 29.08.2018, now sitting in Surrey Space Centre temporarily. 14.12.2018: Tracking ESEO only on UHF, no tracking available for other sats. 08.02.2019: Added ISS for ARISS SSTV Event. 27.06.2019: Tracking ESA ESEO. From Nov 2019, tracking OPS-SAT.","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000},{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":146000000}]",2
254,SP2ZIE - 70cm,Rotator damaged. PW-SAT2 Satellite only. 435.275MHz+/-25kHz bandpass filter.AZ-EL Wimo WX7036. High noise environment. 437-438MHz strong QRMs from LoRa like transmissions. Band limited to 435MHz-440MHz (bandpass filter). Test setup.,"[{""antenna_type"":""cross-yagi"",""antenna_type_name"":""Cross Yagi"",""band"":""UHF"",""frequency"":432000000,""frequency_max"":439000000}]",2
527,Test1,This is only a test for a system later on,"[{""antenna_type"":""helical"",""antenna_type_name"":""Helical"",""band"":""L"",""frequency"":1600000000,""frequency_max"":1800000000},{""antenna_type"":""helical"",""antenna_type_name"":""Helical"",""band"":""L"",""frequency"":1500000000,""frequency_max"":1750000000}]",2
751,VK2BYF,This station is not in use. Created for testing only,[],
959,CE3VRT - VHF - UHF,Station on rural area (limited internet access). Please try to schedule Fox series sats only. Gear:- VHF Turnstile- FM Filter + LNA4ALL- Nooelec RTLSDR,"[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000}]",2
1287,FOSSA-STATION,"Test Tape Measure Yagi, Only North/West Passes","[{""antenna_type"":""yagi"",""antenna_type_name"":""Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":470000000}]",2
1526,Budapest UHF (west only),"2*3el UHF cross yagi facing 270deg AZ 45deg EL. Setup:Orange Pi Zero 256M, LimeSDR Mini, diy filter with ~2dB loss, SPF5043Z ~0.6NF 20dB LNA, diy antenna. Contact me on the community forms if you have any questions.","[{""antenna_type"":""cross-yagi"",""antenna_type_name"":""Cross Yagi"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":440000000}]",2
1594,VK2PET #2 Trial Station,This station is a Trial to play around with antennas & SDR's. It will NOT be online all the time. Limited by myself entering the Satellite passes into PST-Rotator. Back online for testing only as I'm having a lot of issues with this station. For all others please use my other station.,"[{""antenna_type"":""other direct"",""antenna_type_name"":""Other Directional"",""band"":""VHF"",""frequency"":135000000,""frequency_max"":148000000},{""antenna_type"":""other direct"",""antenna_type_name"":""Other Directional"",""band"":""UHF"",""frequency"":430000000,""frequency_max"":450000000}]",2
1687,DL6KBG-AZ

Jackpot!
A few more stations with directional antennas but no rotation.

And a bonus azimuth-only station!

In [16]:
%%sql
-- More no-rotator stations
UPDATE stations SET rotator = 0 WHERE id IN (1526, 1824, 2482);

-- And another azimuth-only station!
UPDATE stations SET rotator = 1 WHERE id IN (2837);

 * sqlite:///../data/stations.db
3 rows affected.
1 rows affected.


[]

That's good enough for now.
Note to self to consider filing an issue against the ground station information endpoint to add a field to capture this number of axes directly from the owner.

What about the stations with an empty `antenna` field?

In [17]:
%%sql
SELECT
    count(*),
    sum(observations)
FROM stations
WHERE rotator NOT IN (0, 1, 2);

 * sqlite:///../data/stations.db
Done.


count(*),sum(observations)
199,47


So there are 199 stations with a blank `antenna` field, which have scheduled a total of only 47 observations across them.
It is not statistically useful to attempt to classify the number of rotation axes for these stations.

# Statistics

## Popular antenna types

In [18]:
%%sql
SELECT
    json_extract(antenna, "$[0].antenna_type") as type,
    count(antenna) as num_type
FROM stations
GROUP BY type
ORDER BY num_type DESC;

 * sqlite:///../data/stations.db
Done.


type,num_type
turnstile,358
yagi,316
quadrafilar,290
vertical,249
cross-yagi,229
None,204
dipole,180
v-dipole,153
discone,140
eggbeater,139


## Number of rotation axes

In [19]:
%%sql
-- Number of stations by motion axes
SELECT
    rotator,
    count(rotator) as num,
FROM stations
GROUP BY rotator
ORDER BY rotator;

 * sqlite:///../data/stations.db
(sqlite3.OperationalError) near "FROM": syntax error
[SQL: -- Number of stations by motion axes
SELECT
    rotator,
    count(rotator) as num,
FROM stations
GROUP BY rotator
ORDER BY rotator;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [20]:
# switch to python
result = _
num_axes = [r[1] for r in result[:3]]
total = sum(num_axes)

print("axes   N   %")
for i, n in enumerate(num_axes):
    print(f"{i:3d}  {n:4d} {n/total*100:4.1f}")

axes   N   %
  0   358 37.1
  1   316 32.8
  2   290 30.1


Of the registered ground stations, the ratio of locations with no rotator versus some rotator is about 70 % to 30 %.

What if we restrict this to stations which are presently operating as either "Online" or "Testing"?

In [21]:
%%sql
-- Number of stations by motion axes
SELECT
    rotator,
    count(rotator) as num
FROM stations
WHERE status IN ("Online", "Testing")  -- Only other option is "Offline"
GROUP BY rotator
ORDER BY rotator;

 * sqlite:///../data/stations.db
Done.


rotator,num
0,307
1,2
2,90
,1


In [22]:
result = _
num_axes = [r[1] for r in result[:3]]
total = sum(num_axes)

print("axes   N   %")
for i, n in enumerate(num_axes):
    print(f"{i:3d}  {n:4d} {n/total*100:4.1f}")

axes   N   %
  0   307 76.9
  1     2  0.5
  2    90 22.6


The ratio increases in favor of no rotator for the antenna (or merely omni-directional).

## Online status

In [23]:
%%sql
-- Number of stations by motion axes
SELECT
    status,
    count(status) as num
FROM stations
GROUP BY status
ORDER BY num DESC;

 * sqlite:///../data/stations.db
Done.


status,num
Offline,2204
Online,263
Testing,137


In [24]:
result = _
total = sum([r[1] for r in result])

for s, n in result:
    print(f"{s:8s} {n:4d} {n/total*100:4.1f}%")

Offline  2204 84.6%
Online    263 10.1%
Testing   137  5.3%


About 15% of registered ground stations are presently operating and ready to accept observation jobs.

## Ground stations created by year

In [25]:
%%sql
SELECT
    strftime("%Y", created) as year,
    count(id) as num_new
FROM stations
WHERE strftime("%s", created) >= strftime("%s", printf("%d-01-01", year))
  AND strftime("%s", created) <  strftime("%s", printf("%d-01-01", year+1))
GROUP BY year
ORDER BY year;


 * sqlite:///../data/stations.db
Done.


year,num_new
2015,4
2016,1
2017,25
2018,349
2019,700
2020,608
2021,498
2022,419
